In [1]:
from collections import defaultdict

import numpy as np
import pandas as pd
import warnings

from surprise import Dataset, Reader
from tqdm import tqdm

from src.models import cf

tqdm.pandas()
warnings.filterwarnings('ignore')

/Users/jensen/Desktop/recommendation-systems/projects/product-reviews-recommender/.venv/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


# Load Data

In [2]:
# global variables
DATA_PATH = "data/evaluation"
CATEGORY = "Clothing_Shoes_and_Jewelry"

train = pd.read_csv(f"{DATA_PATH}/{CATEGORY}_train.csv")

In [3]:
# checking train dataframe
train.head().append(train.tail())

,index,asin,title,categories,reviewerID,overall,reviewText,reviewTime,processedReviewText
0,0,0000031887,Ballet Dress-Up Fairy Tutu,"[['Clothing, Shoes & Jewelry', 'Girls', 'Cloth...",A1KLRMWW2FWPL4,5.0,This is a great tutu and at a really great pri...,2011-02-12,great tutu great price look cheap glad look am...
1,1,0000031887,Ballet Dress-Up Fairy Tutu,"[['Clothing, Shoes & Jewelry', 'Girls', 'Cloth...",A2G5TCU2WDFZ65,5.0,I bought this for my 4 yr old daughter for dan...,2013-01-19,buy yr old daughter dance class wear today tim...
2,2,0000031887,Ballet Dress-Up Fairy Tutu,"[['Clothing, Shoes & Jewelry', 'Girls', 'Cloth...",A1RLQXYNCMWRWN,5.0,What can I say... my daughters have it in oran...,2013-01-04,daughter orange black white pink think buy fuc...
3,3,0000031887,Ballet Dress-Up Fairy Tutu,"[['Clothing, Shoes & Jewelry', 'Girls', 'Cloth...",A8U3FAMSJVHS5,5.0,"We bought several tutus at once, and they are ...",2014-04-27,buy tutu high review sturdy seemingly girl wea...
4,4,0000031887,Ballet Dress-Up Fairy Tutu,"[['Clothing, Shoes & Jewelry', 'Girls', 'Cloth...",A3GEOILWLK86XM,5.0,Thank you Halo Heaven great product for Little...,2014-03-15,thank halo heaven great product little girls g...
176500,278257,B00JULS24Q,Buyinhouse Feeling Adorable Cute Pink Bowknot ...,"[['Clothing, Shoes & Jewelry', 'Novelty, Costu...",A2IQT5AFFXA1OM,4.0,This item is exactly as described but for some...,2014-05-21,item exactly described reason like person like...
176501,278290,B00K035Y08,Sakkas 197 Oasis Gauzy Crepe Sleeveless Blouse...,"[['Clothing, Shoes & Jewelry', 'Women', 'Cloth...",A1XWMGHBAPKOV3,5.0,Put on and it fit great easy care also.Would t...,2014-06-15,fit great easy care tell friend buy thank hawa...
176502,278297,B00K0352PU,Sakkas Paradise Embroidered Relaxed Fit Blouse,"[['Clothing, Shoes & Jewelry', 'Women', 'Cloth...",A3VTQB69FYGQDU,3.0,Before I washed the top it was extremely large...,2014-06-06,wash extremely large fit perfectly wash embroi...
176503,278316,B00K8J06CK,TrendzArt Azules Poly Span Floral Print Full L...,"[['Clothing, Shoes & Jewelry', 'Women', 'Cloth...",A241BLSJL8AGY,4.0,This maxi skirt was very nice material it fit ...,2014-06-20,maxi skirt nice material fit nicely color pret...
176504,278345,B00KF9180W,[2 PACK] Multi-Purpose Sports Balaclava - For ...,"[['Clothing, Shoes & Jewelry', 'Men', 'Accesso...",A2YKWYC3WQJX5J,5.0,This is truly a year round product. Here in th...,2014-06-21,truly year round product midwest summer use ba...


# Preparing Dataset for Surprise's Algorithm

In [4]:
# create reader 
reader = Reader(rating_scale=(1,5))

# Utility Functions

In [5]:
def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in tqdm(predictions):
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in tqdm(top_n.items()):
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

def recall_at_k(asins, predicted_asins, k=10):
    # number of relevant items
    set_actual = set(asins)
    set_preds = set(predicted_asins)
    num_relevant = len(set_actual.intersection(set_preds))
    
    # calculating recall@K - relevant / total relevant items
    recall_at_k = num_relevant / len(asins)
    
    return recall_at_k

def novelty_at_k(item_popularity, predicted_asins, k=10):
    """
    """
    # finding avg novelty
    popularity_sum = item_popularity.loc[predicted_asins].sum()
    novelty_at_k = ((k*1) - popularity_sum) / k
    
    return novelty_at_k

def generate_item_popularity(train: pd.DataFrame) -> pd.DataFrame:
    """
    """
    
    # create a mapping of item popularatity
    # based on sum(item's review / max reviews) / no items
    max_reviews = (train.groupby(['asin'])
                   .agg({'processedReviewText': 'count'})
                   .max()
                   .values[0])
    item_popularity = (train.groupby(['asin'])
                       .agg({'processedReviewText': 'count'})
                       .apply(lambda x: x/max_reviews))
    
    return item_popularity
    

def evaluate_recommendations(top_ns: dict, user_rating_history: pd.DataFrame, item_popularity: pd.DataFrame, k=10) -> pd.DataFrame:
    """
    
    Args:
        top_ns
        user_rating_history
    """
    
    test_recommendations = pd.DataFrame(top_ns.items(), columns=["reviewerID", "pred_asin"])
    test_recommendations['pred_asin'] = test_recommendations['pred_asin'].apply(lambda x: [i[0] for i in x])
    
    # combined test history and recommendations
    test_merged = pd.merge(user_rating_history, test_recommendations, on="reviewerID", how="inner")
    
    # generating recall@k metrics
    test_merged["recall@k"] = test_merged.apply(lambda x: recall_at_k(x.asin, x.pred_asin, k=k), axis=1)
    test_merged["novelty@k"] = test_merged.apply(lambda x: novelty_at_k(item_popularity, x.pred_asin, k=k), axis=1)
    average_recall_at_k = test_merged["recall@k"].mean()
    average_novelty_at_k = test_merged["novelty@k"].mean()
    
    print(f"The MEM-ECF has an average recall@{k}: {average_recall_at_k:.5f}, average novelty@{k}: {average_novelty_at_k:.5f}")
    
    return test_merged

# Generate N-Recommendations = {10, 25, 30, 45}

## Load Test Data

In [6]:
test = pd.read_csv(f"{DATA_PATH}/{CATEGORY}_test.csv")

In [7]:
test.head().append(test.tail())

,index,asin,title,categories,reviewerID,overall,reviewText,reviewTime,processedReviewText
0,6,0000031887,Ballet Dress-Up Fairy Tutu,"[['Clothing, Shoes & Jewelry', 'Girls', 'Cloth...",A16GFPNVF4Y816,5.0,Bought this as a backup to the regular ballet ...,2014-05-03,bought backup regular ballet outfit daughter w...
1,17,0000031887,Ballet Dress-Up Fairy Tutu,"[['Clothing, Shoes & Jewelry', 'Girls', 'Cloth...",A2XJ13PIXVJFJH,1.0,Never GOT this item - but gave a 1 STAR becaus...,2014-05-12,item star reply supplier great try send item r...
2,23,0123456479,SHINING IMAGE HUGE PINK LEATHER JEWELRY BOX / ...,"[['Clothing, Shoes & Jewelry', 'Novelty, Costu...",A2WNN1DQVL4LH5,5.0,The minute I saw this my heart skipped a beat....,2013-11-07,minute saw heart skip beat nice case sort coll...
3,24,0123456479,SHINING IMAGE HUGE PINK LEATHER JEWELRY BOX / ...,"[['Clothing, Shoes & Jewelry', 'Novelty, Costu...",A1ZPOCG2ST2CY3,5.0,Love this Jewelry Box so well put together ho...,2014-01-19,love jewelry box hold plendy love pink look ni...
4,27,0123456479,SHINING IMAGE HUGE PINK LEATHER JEWELRY BOX / ...,"[['Clothing, Shoes & Jewelry', 'Novelty, Costu...",A1JC50F14SLAEV,3.0,I wanted to have the title summarize my though...,2014-05-12,want title summarize thought decide read entir...
99497,278341,B00KF9180W,[2 PACK] Multi-Purpose Sports Balaclava - For ...,"[['Clothing, Shoes & Jewelry', 'Men', 'Accesso...",A1EVV74UQYVKRY,4.0,I go walking a lot in all kinds of weather and...,2014-06-16,walk lot kind weather know lot like block cold...
99498,278342,B00KF9180W,[2 PACK] Multi-Purpose Sports Balaclava - For ...,"[['Clothing, Shoes & Jewelry', 'Men', 'Accesso...",ABUE0ALHKWKHC,5.0,This two pack of Balaclavas makes for a very n...,2014-06-09,pack balaclava nice purchase balaclava fit snu...
99499,278343,B00KF9180W,[2 PACK] Multi-Purpose Sports Balaclava - For ...,"[['Clothing, Shoes & Jewelry', 'Men', 'Accesso...",A1PI8VBCXXSGC7,5.0,"Well, the first thing I did was try the balacl...",2014-06-13,thing try balaclava hubby try fit average size...
99500,278344,B00KF9180W,[2 PACK] Multi-Purpose Sports Balaclava - For ...,"[['Clothing, Shoes & Jewelry', 'Men', 'Accesso...",A2XX2A4OJCDNLZ,5.0,While balaclavas can be used for a variety of ...,2014-06-13,balaclava use variety thing use mainly late fa...
99501,278346,B00KF9180W,[2 PACK] Multi-Purpose Sports Balaclava - For ...,"[['Clothing, Shoes & Jewelry', 'Men', 'Accesso...",A3UJRNI8UR4871,4.0,"Nice material, but not as nice as silk or mer...",2014-06-09,nice material nice silk merino wool course mat...


In [8]:
# generating test history
test_user_history = (pd.DataFrame(test.groupby(['reviewerID'])['asin']
                                  .apply(list).reset_index()))

In [9]:
print(test_user_history)

                  reviewerID                                              asin
0      A001114613O3F18Q5NVR6              [B000J6ZYL0, B005BXP7R2, B0093STGGO]
1      A00146182PNM90WNNAZ5Q              [B006Y4QDVQ, B00823Y41S, B00BQJV1LG]
2      A00165422B2GAUE3EL6Z0                          [B008SBGKP2, B00BLW2PZY]
3      A00338282E99B8OR2JYTZ                          [B003F8BKGW, B00DVFNNQE]
4      A00354001GE099Q1FL0TU                          [B0058YTOP0, B00BTWAZ0I]
...                      ...                                               ...
39358          AZZMQ85DPFEG3  [B000S7O8AS, B009N0CSXU, B00CKGB85I, B00DUW4VAA]
39359          AZZNK89PXD006                          [B004L7J7IO, B008ZBPQJ6]
39360          AZZT1ERHBSNQ8              [B00856U6BE, B0089GNZNQ, B00CPK44DM]
39361          AZZTOUKVTUMVM                          [B000VL04LS, B0053XF2U2]
39362          AZZYW4YOE1B6E                          [B007UNSF3O, B008J4RESK]

[39363 rows x 2 columns]


# Instantiate FunkSVD (Matrix Factorization)

In [10]:
# instantiating funksvd 
funk_svd = cf.FunkMF(n_epochs=5, lr_all=0.005)

In [11]:
# fitting to the training data
funk_svd.fit(train)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4


In [12]:
%%time
# creating testset for prediction
testset = funk_svd.trainset.build_anti_testset()

CPU times: user 9min 4s, sys: 5min, total: 14min 5s
Wall time: 18min 15s


In [ ]:
%%time
# generate candidate times
candidate_items = funk_svd.test(testset)

## Loop through N = {10, 25, 30, 45}

In [ ]:
# generate item popularity
item_popularity = generate_item_popularity(train)

In [ ]:
n_recommendations = {}
for n in [10, 25, 30, 45]:
    # retrieve the top-n items based on similarities
    top_ns = get_top_n(candidate_items, n)
    # evaluate how well the recommended items predicted the future purchases
    n_recommended_items = evaluate_recommendations(top_ns, test_user_history, item_popularity, n)
    # saving the n-value and recommended items
    n_recommendations[n] = (top_ns, n_recommended_items)

# Evaluate N-Recommendations

In [ ]:
def retrieve_recommendations(train: pd.DataFrame, top_ns: dict):
    """
    """
    # generating a random user
    random_user = np.random.choice(list(train['reviewerID'].unique()), 1)[0]
    print(f"For user: {random_user}:")
    print(f"Purchase History:\n{train[train['reviewerID'] == random_user][['asin', 'title']]}")

    # find the recommendations
    print(f"\nRecommending:\n")
    recommendations = (train[train['asin']
                             .isin([i[0] for i in top_ns[random_user]])][['asin', 'title']]
                       .drop_duplicates(subset='asin')
                       .set_index('asin'))
    print(f"{recommendations.loc[[i[0] for i in top_ns[random_user]]].reset_index()}")

## N=10

In [ ]:
top_ns_10 = n_recommendations[10][0]
retrieve_recommendations(train, top_ns_10)

## N=25

In [ ]:
top_ns_25 = n_recommendations[25][0]
retrieve_recommendations(train, top_ns_25)

## N=30

In [ ]:
top_ns_30 = n_recommendations[30][0]
retrieve_recommendations(train, top_ns_30)

## N=45

In [ ]:
top_ns_45 = n_recommendations[45][0]
retrieve_recommendations(train, top_ns_45)